In [110]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize
import requests
from amadeus import Location
from amadeus import Client, ResponseError

In [111]:
amadeus = Client(
    client_id='mfWDFAuJHzCAyiuyrF2ZNPLt86x6pBHe',
    client_secret='20kJCr29aZVItX2F'
)

try:
    response = amadeus.reference_data.urls.checkin_links.get(airlineCode='BA')
    print(response.data)
except ResponseError as error:
    print(error)

[{'type': 'checkin-link', 'id': 'BAEN-GBAll', 'href': 'https://www.britishairways.com/travel/olcilandingpageauthreq/public/en_gb', 'channel': 'All'}, {'type': 'checkin-link', 'id': 'BAEN-GBMobile', 'href': 'https://www.britishairways.com/travel/managebooking/public/en_gb/device-mobile?bookingRef={PNR}&lastname={LAST}', 'channel': 'Mobile', 'parameters': {'LAST': {'description': 'Passenger Last Name', 'type': 'string'}, 'PNR': {'description': 'Record Locator', 'type': 'string', 'format': '([a-zA-Z]|[0-9]){6}'}}}, {'type': 'checkin-link', 'id': 'BAEN-GBWeb', 'href': 'https://www.britishairways.com/travel/managebooking/public/en_gb?&bookingRef={PNR}&lastname={LAST}', 'channel': 'Web', 'parameters': {'LAST': {'description': 'Passenger Last Name', 'type': 'string'}, 'PNR': {'description': 'Record Locator', 'type': 'string', 'format': '([a-zA-Z]|[0-9]){6}'}}}]


## Where did most people fly to from SFO in Valentineds day 2018? (from Feb 01 to Feb 28 - full month)

In [112]:
response_best_place = amadeus.travel.analytics.air_traffic.traveled.get(originCityCode='SFO', period='2018-02')
response_best_place = response_best_place.data
print(response_best_place)

[{'type': 'air-traffic', 'destination': 'LAS', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 63}, 'travelers': {'score': 100}}}, {'type': 'air-traffic', 'destination': 'LAX', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 86}, 'travelers': {'score': 93}}}, {'type': 'air-traffic', 'destination': 'NYC', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 100}, 'travelers': {'score': 91}}}, {'type': 'air-traffic', 'destination': 'CHI', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 52}, 'travelers': {'score': 36}}}, {'type': 'air-traffic', 'destination': 'BLR', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 17}, 'travelers': {'score': 13}}}, {'type': 'air-traffic', 'destination': 'MAD', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 27}, 'travelers': {'score': 11}}}, {'type': 'air-traffic', 'destination': 'SIN', 'subType': 'TRAVELED', 'analytics': {'flights': {'score': 14}, 'travelers': {'score': 10}}}, {'type': 'air-traffic', 

In [113]:
# print(json.dumps(response, indent=4, sort_keys=True))
print(json.dumps(response_best_place, indent=4, sort_keys=True))

[
    {
        "analytics": {
            "flights": {
                "score": 63
            },
            "travelers": {
                "score": 100
            }
        },
        "destination": "LAS",
        "subType": "TRAVELED",
        "type": "air-traffic"
    },
    {
        "analytics": {
            "flights": {
                "score": 86
            },
            "travelers": {
                "score": 93
            }
        },
        "destination": "LAX",
        "subType": "TRAVELED",
        "type": "air-traffic"
    },
    {
        "analytics": {
            "flights": {
                "score": 100
            },
            "travelers": {
                "score": 91
            }
        },
        "destination": "NYC",
        "subType": "TRAVELED",
        "type": "air-traffic"
    },
    {
        "analytics": {
            "flights": {
                "score": 52
            },
            "travelers": {
                "score": 36
            }
     

# Take the top 3 destination airports and we query the price of the flights for same date range in 2019

# Store JSON data into a DataFrame

In [114]:
#json_file = "../Resources/customer_location.json"
best_places_df = pd.DataFrame.from_dict(json_normalize(response_best_place), orient='columns')
best_places_df

,analytics.flights.score,analytics.travelers.score,destination,subType,type
0,63,100,LAS,TRAVELED,air-traffic
1,86,93,LAX,TRAVELED,air-traffic
2,100,91,NYC,TRAVELED,air-traffic
3,52,36,CHI,TRAVELED,air-traffic
4,17,13,BLR,TRAVELED,air-traffic
5,27,11,MAD,TRAVELED,air-traffic
6,14,10,SIN,TRAVELED,air-traffic
7,21,7,BCN,TRAVELED,air-traffic
8,11,7,SNA,TRAVELED,air-traffic
9,17,5,IST,TRAVELED,air-traffic


# Clean DataFrame

## TRANSFORM: GET RID OF COLUMNS WE DONT NEED FROM THE JSON CALL

In [160]:
# Get a specific city or airport based on its id

airport_codes = best_places_df['destination'].tolist()
# airport_codes

test_list = []

for airport in airport_codes:
    try:
        response = amadeus.reference_data.locations.get(
        keyword=airport,
        subType=Location.ANY)

     #  print(airport) 

        test_list.append(response.data[0]['address']['countryName'])

    except:
        test_list.append('not here')
    

best_places_df['country'] = pd.DataFrame(test_list)

In [161]:
best_places_df

,analytics.flights.score,analytics.travelers.score,destination,subType,type,country
0,63,100,LAS,TRAVELED,air-traffic,UNITED STATES OF AMERICA
1,86,93,LAX,TRAVELED,air-traffic,UNITED STATES OF AMERICA
2,100,91,NYC,TRAVELED,air-traffic,UNITED STATES OF AMERICA
3,52,36,CHI,TRAVELED,air-traffic,UNITED STATES OF AMERICA
4,17,13,BLR,TRAVELED,air-traffic,INDIA
5,27,11,MAD,TRAVELED,air-traffic,SPAIN
6,14,10,SIN,TRAVELED,air-traffic,INDIA
7,21,7,BCN,TRAVELED,air-traffic,SPAIN
8,11,7,SNA,TRAVELED,air-traffic,UNITED STATES OF AMERICA
9,17,5,IST,TRAVELED,air-traffic,not here


In [154]:
# print(json.dumps(response.data, indent=4, sort_keys=True))
print(response.data[0]['address']['countryName'])

IndexError: list index out of range

# Connect to local database

In [ ]:

# rds_connection_string = "postgres:{PASSWORD}-@localhost:5432/{DATABASE NAME}"
# engine = create_engine(f'postgresql://{rds_connection_string}')

## Check for tables

In [ ]:
# engine.table_names()

# Use pandas to load json converted DataFrame into database

In [ ]:
# Flight Most Traveled Destinations Where were people flying to from Madrid in the January 2017? 
# GET/v1/travel/analytics/air-traffc/traveled ?originCityCode=MAD&period=2017-01

In [ ]:
# Flight Low-fare Search I know where I want to fly, the dates and duration, what are the best flight deals?
# GET/v1/shopping/fight-offers ?origin=PAR&destination=LON&departureDate=2019-07-25&returnDate=2019-07-28

In [ ]:
# Flight Inspiration Search
amadeus.shopping.flight_destinations.get(origin='MAD')

# Flight Cheapest Date Search
amadeus.shopping.flight_dates.get(origin='MAD', destination='MUC')

# Flight Low-fare Search
amadeus.shopping.flight_offers.get(origin='MAD', destination='NYC', departureDate='2019-08-01')

# Flight Choice Prediction
result = amadeus.shopping.flight_offers.get(origin='MAD', destination='NYC', departureDate='2019-08-01').result
amadeus.shopping.flight_offers.prediction.post(result)

# Flight Checkin Links
amadeus.reference_data.urls.checkin_links.get(airlineCode='BA')

# Airline Code Lookup
amadeus.reference_data.airlines.get(airlineCodes='U2')

# Airport and City Search (autocomplete)
# Find all the cities and airports starting by 'LON'
amadeus.reference_data.locations.get(keyword='LON', subType=Location.ANY)
# Get a specific city or airport based on its id
amadeus.reference_data.location('ALHR').get()

# Airport Nearest Relevant Airport (for London)
amadeus.reference_data.locations.airports.get(longitude=0.1278, latitude=51.5074)

# Flight Most Searched Destinations
# Which were the most searched flight destinations from Madrid in August 2017?
amadeus.travel.analytics.air_traffic.searched.get(originCityCode='MAD', marketCountryCode='ES', searchPeriod='2017-08')
# How many people in Spain searched for a trip from Madrid to New-York in September 2017?
amadeus.travel.analytics.air_traffic.searched_by_destination.get(originCityCode='MAD', destinationCityCode='NYC', marketCountryCode='ES', searchPeriod='2017-08')

# Flight Most Booked Destinations
amadeus.travel.analytics.air_traffic.booked.get(originCityCode='MAD', period='2017-08')

# Flight Most Traveled Destinations
amadeus.travel.analytics.air_traffic.traveled.get(originCityCode='MAD', period='2017-01')

# Flight Busiest Travel Period
amadeus.travel.analytics.air_traffic.busiest_period.get(cityCode='MAD', period='2017', direction='ARRIVING')

# Hotel Search
# Get list of Hotels by city code
amadeus.shopping.hotel_offers.get(cityCode = 'LON')
# Get list of offers for a specific hotel
amadeus.shopping.hotel_offers_by_hotel.get(hotelId = 'IALONCHO')
# Confirm the availability of a specific offer
amadeus.shopping.hotel_offer('D5BEE9D0D08B6678C2F5FAD910DC110BCDA187D21D4FCE68ED423426D0A246BB').get()

# Point of Interest
# What are the popular places in Barcelona (based a geo location and a radius)
amadeus.reference_data.locations.points_of_interest.get(latitude=41.397158, longitude=2.160873)
# What are the popular places in Barcelona? (based on a square)
amadeus.reference_data.locations.points_of_interest.by_square.get(north=41.397158, west=2.160873, south=41.394582, east=2.177181)